### Load Packages

In [ ]:
import pandas as pd
import numpy as np
import os

pd.options.mode.chained_assignment = None

### Import Excel Files

In [ ]:
folder = r'C:\Users\mazhang\Documents\Jupyter Notebook\2021 Half-Monthly Closed Invoice\Closed_invoice_012021_092021'
files = os.listdir(folder)

excel_list = []
for file in files:
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder, file), sheet_name = None)
        excel_list.append(df)

### Generate Half-Monthly Report with Accounting ID, Facility and  Bill Pay Category, etc.

In [ ]:
df_list = []

for excel in range(len(excel_list)):
    
    df_header = excel_list[excel]['Header']
    df_line = excel_list[excel]['Line']
    df_detail = excel_list[excel]['Detail']

    df_select_header = df_header[['Accounting_CustomerID', 'CustomerID', 'Facility', 'Bill To']]
    df_select_header['Billing Period Start'] = df_detail['Header Billing Period Start']
    df_drop_dup_header = df_select_header.drop_duplicates(subset = ['Accounting_CustomerID', 'Facility', 'Bill To'], 
                                                          keep = 'last')
    
    df_select_line = df_line[['CustomerID', 'Facility', 'Bill Pay Category', 'Bill To']]
    df_drop_dup_line = df_select_line.drop_duplicates(subset = ['Facility', 'Bill To', 'Bill Pay Category'], 
                                                      keep = 'last')

    df_combine_header_line = pd.merge(df_drop_dup_line, df_drop_dup_header, how = 'left', 
                                      on = ['Facility', 'Bill To']).T.drop_duplicates().T
    df_combine_rename = df_combine_header_line.rename(columns = {"CustomerID_x": "CustomerID"})
    df_combine_half_month = df_combine_rename.reindex(columns = ['Accounting_CustomerID', 'CustomerID', 'Facility', 
                                                                 'Bill Pay Category', 'Bill To', 'Billing Period Start'])
    print(df_combine_half_month.shape)
    
    df_list.append(df_combine_half_month)

### Generate Quarter Report with Accounting ID, Facility and  Bill Pay Category, etc.

In [ ]:
df_combine_list = []
df_combine_quarter = df_list[0]

for df in range(len(df_list)):
    
    df_combine_quarter = pd.merge(df_combine_quarter, df_list[df], how = 'outer', 
                                  on = ['Accounting_CustomerID', 'CustomerID', 'Facility', 'Bill Pay Category', 'Bill To'])
    print(df_combine_quarter.shape)
    
    df_combine_list.append(df_combine_quarter)
    
df_output_init = df_combine_list[-1]
df_output = df_output_init.T.drop_duplicates().T
df_output.rename({'Billing Period Start_x':'Billing Period Start', 
                'Billing Period Start_y':'Billing Period Start'}, axis = 'columns', inplace = True)
print(df_output.shape)

### Export Quarter Report As Excel File

In [ ]:
df_output.to_excel("Quarter Report.xlsx", index = False, header = True)

# Method 1 For Convert

### Convert Date Value to 0/1 Value

In [ ]:
name1 = np.array(df_output.columns[0:5])
name2 = df_output.iloc[:, 5:].dropna().iloc[0].values
name = np.concatenate((name1, name2))
df1 = df_output.copy()
df1.columns = name
df1.head()

In [ ]:
name_rp = np.concatenate((name, np.array(['09/21/2021', '09/22/2021'])))

In [ ]:
df2 = df1.replace(to_replace=name_rp , value=1).fillna(0)
df2

In [ ]:
# df2.to_excel("Quarter Report 0_1 new.xlsx", index = False, header = True)

### Extract Only Billed Once Account

In [ ]:
df3 = df_output.loc[df2.sum(axis=1) < 2]
df3

In [ ]:
# df3.to_excel("Quarter Report Only billed Once.xlsx", index = False, header = True)

# Method 2 For Convert (better)

### Convert Date Value to 0/1 Value

In [ ]:
df0 = df_output.iloc[:, 6:].fillna(0)
df0

In [ ]:
uni_date_int = np.array(pd.unique(df0[['Billing Period Start']].values.ravel('K')))
uni_date = np.delete(uni_date_int, np.argwhere(uni_date_int == 0))
uni_date

In [ ]:
df1 = df_output.copy()
df2 = df1.replace(to_replace = uni_date , value = 1).fillna(0)
df2

### Extract Only Billed Once Account

In [ ]:
df3 = df_output.loc[df2.sum(axis = 1) < 2]
df3

In [ ]:
# df3.to_excel("Quarter Report Only billed Once.xlsx", index = False, header = True)